In [1]:
import os
import fnmatch

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from lr_video_face.plots import *

from lir import Xy_to_Xn, metrics

In [2]:

home = os.path.expanduser("~")

# SQL database path and name
input_dir = os.path.join(home, 'video_resources', 'sql_database_21')
#db_name = 'colab_main_dataset'
# Where results are saved
output_dir = os.path.join(home, 'video_resources', 'output_4')

#Traducción de nombres
dic_emb = {'Facenet512': 'FaceNet', 'QMagFace':'QMagFace', 'ArcFace':'ArcFace'}
dic_qua = {'tface' : 'SDD-FIQA','ser_fiq': 'SER-FIQ'}

In [3]:

files = []
for root, dir, filenames in os.walk(output_dir):
    for filename in fnmatch.filter(filenames, 'dat*.pd'):
        files.append(os.path.join(root, filename))
   

In [4]:
def generate_plot(file):

    df0 = pd.read_pickle(file)

    if len(pd.unique(df0.Detector)) == 1 and len(pd.unique(df0.Calibrator)) == 1:

        cols = list(pd.unique(df0['Embedding Model']))
        rows = list(pd.unique(df0['Quality Model']))

        #generamos la gráfica con subplots
        fig,ax = plt.subplots(nrows = len(rows), ncols = len(cols), squeeze= False, figsize= (25,10))
        
        for index, df1 in df0.iterrows():

            col = cols.index(df1['Embedding Model'])
            row = rows.index(df1['Quality Model'])

            ax1 = ax[row][col]
            #ax1.ylabel = df1['Embedding Model']
            #ax1.set_title ( f"Quality Model: {dic_qua [df1['Quality Model']]}")
            ax1.set_title ( f"Embedding Model: {dic_emb [df1['Embedding Model']]}",fontsize = 14)

            subplot_new(ax1,df1.Results, df1.Cllr)
            #ax1.set(ylabel= f"Embedding Model: {dic_emb[df1['Embedding Model']]}\n Cllr")
            
            #ax1.set(ylabel= f"Quality Model: {dic_qua[df1['Quality Model']]}\n Cllr")
            ax1.set_ylabel(f"Quality Model: {dic_qua[df1['Quality Model']]}\n Cllr",fontsize = 14)
            ax1.set_xlabel('% of discarded pairs',fontsize=14)
        plt.suptitle('$C_{llr}$ Values',fontsize=18)

        #leyenda global
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=14, borderaxespad=0.)
        fig.tight_layout()
        #savefig = os.path.join(self.experiments.output_dir, f"cllr_summary_ESX{self.experiments.embedding_model_as_scorer}")
        savefig = os.path.join(os.path.dirname(file), os.path.splitext(file)[0]+'_v4')
        plt.savefig(savefig, dpi=600)
        plt.close() 

In [5]:
def generate_latex(file):
    df0 = pd.read_pickle(file)


    
    

    if len(pd.unique(df0.Detector)) == 1 and len(pd.unique(df0.Calibrator)) == 1:
        df_latex = pd.DataFrame()

        for index, df1 in df0.iterrows():    

            emb_model = df1['Embedding Model']
            qua_model = df1['Quality Model']
            # the results are only received for 2015
            lrs_predicted = df1.Results["lrs_predicted"]
            y_test = df1.Results["y_test"]
            dropouts = df1.Results["quality_drops"]
            common_attributes = df1.Results['common_attributes']
    
            cllr_participants = np.mean(df1.Cllr)

            # imagen promedio (17 comparisons)
            lr_avg = df1.Results["lrs_predicted_2015"]
            y_avg = df1.Results["y_test_2015"]    
    
            cllr_avg = metrics.cllr(np.asarray(lr_avg), np.asarray(y_avg)) 
            df_latex = df_latex.append({'Embedding_model': 'Participants',
                                            'Quality_model': dic_qua[qua_model],
                                             'Cllr': cllr_participants},ignore_index = True)
            df_latex = df_latex.append({'Embedding_model': 'Average Image',
                                            'Quality_model': dic_qua[qua_model],
                                             'Cllr': cllr_avg},ignore_index = True)

            #get cllr per dropout

            
            for d in set(dropouts):
                lr_d = [lr for lr,dropout in zip(lrs_predicted,dropouts) if dropout== d ]
                y_d = [y for y,dropout in zip(y_test,dropouts) if dropout== d ]
                cllr_d = metrics.cllr(np.asarray(lr_d), np.asarray(y_d))

            #solo cambio en el momento de plotear
                df_latex = df_latex.append({'Embedding_model': dic_emb[emb_model],
                                            'Quality_model': dic_qua[qua_model],
                                             'Cllr': cllr_d,
                                             'Dropout': 100*(1-d)},ignore_index = True)

            df_latex.sort_values(by= 'Dropout', inplace = True)
            #df_latex.dropna(inplace= True)

            

            # get cllr per common attributes
            # filter results when dropout = 1 
            lrs = [lr for lr,drop in zip(lrs_predicted,dropouts) if drop== 1] 
            ys = [y for y,drop in zip(y_test,dropouts) if drop== 1]
            comatt = [ca for ca,drop in zip(common_attributes,dropouts) if drop== 1]


            #df_plot2 = pd.DataFrame()
            for c in set(comatt):
                lr_c = [lr for lr,com in zip(lrs,comatt) if com== c ]
                y_c = [y for y,comm in zip(ys,comatt) if comm== c ]
                cllr_c = metrics.cllr(np.asarray(lr_c), np.asarray(y_c))
                df_latex = df_latex.append({'Embedding_model': dic_emb[emb_model],
                                            'Quality_model': dic_qua[qua_model],
                                             'Cllr': cllr_c,
                                             'Common Attributes': c},ignore_index = True)
        
            df_latex.sort_values(by= 'Common Attributes', inplace = True)
            #df_latex.dropna(inplace= True)   

#----
        latex_table = df_latex.to_latex(index=False)
        savelatex = os.path.join(os.path.dirname(file), os.path.splitext(file)[0]+'.tex')

        with open (savelatex,'w') as f:
            f.write(latex_table)

In [6]:
for file in files:
    generate_plot(file)
    generate_latex(file)


0.9140601307433852 [0.30709334 0.478332   1.2339668 ]
0.9784160900002613 [0.38559002 0.65506112 1.24964784]
0.9512250947788379 [0.34394204 0.58066824 1.25335868]
1.0161309968455097 [0.45745785 0.80329372 1.25891104]
1.0017104291518084 [0.42023153 0.74267341 1.25853407]
1.0277636889498645 [0.49577808 0.85039098 1.26713581]
1.045036864769775 [0.53260249 0.89324275 1.27940619]
1.0572878347126942 [0.56994625 0.92731717 1.2961782 ]
1.0653718887437598 [0.60834122 0.94877683 1.30929094]
1.0733025609947737 [0.64195318 0.9681684  1.32508164]


/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The fram

1.0181198893915557 [0.29002658 0.63915504 1.35268857]
1.0516213740316853 [0.36164313 0.77298083 1.40435119]
1.0380996171531458 [0.3173193  0.68774605 1.3475903 ]
1.117021105721689 [0.42996357 0.89427812 1.54803516]
1.0912747311296371 [0.39112201 0.83503825 1.48646572]
1.1252315274580784 [0.45904128 0.93786047 1.57562069]
1.12856150679836 [0.4762888  0.9525481  1.58797294]
1.1424295643237534 [0.49194035 0.96771353 1.61102835]
1.1520858830680885 [0.51415204 0.98326327 1.63469188]
1.1551727239928096 [0.52497854 0.98620851 1.64369878]


/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The fram

0.6785614681122885 [0.18354091 0.32652399 0.92562507]
0.7427958872567616 [0.25546134 0.47800183 0.94924705]
0.7170925810807058 [0.22328875 0.40373851 0.93743732]
0.7897813230769527 [0.3234562  0.63869127 1.00466978]
0.7659155336844162 [0.28437661 0.5531251  0.9742072 ]
0.825366875587251 [0.36991647 0.72034937 1.04825498]
0.8553793524047686 [0.4141772  0.7771231  1.08193614]
0.8844366569505946 [0.4605336  0.82411409 1.11030352]
0.9042636532540846 [0.50497842 0.85406746 1.1300006 ]
0.9202385225346619 [0.54736808 0.88543473 1.14348737]


...

/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:458: FutureWarning: The fram

0.7071268916756964 [0.47823969 0.67451489 0.88469457]
0.7644695010502369 [0.50624553 0.70010495 1.03513115]
0.7322164781789339 [0.50015066 0.68279137 0.95022164]
0.8421652883651904 [0.54175033 0.78838788 1.33280013]
0.8035511130878756 [0.52844312 0.74024822 1.17037233]
0.87244925383633 [0.55515732 0.83175436 1.40786438]
0.8832437117005133 [0.55667484 0.84746983 1.43517808]
0.903091401509149 [0.56640126 0.88888375 1.48280389]
0.9178211339007747 [0.57235013 0.92409341 1.51330475]
0.9298592815497848 [0.57446893 0.94497118 1.53628316]
1.5207177968032786 [0.18228077 2.5989525  3.0180207 ]
1.6197409314220108 [0.16580913 2.85827504 3.25396453]
1.5809980092147222 [0.17397552 2.7318348  3.18465345]
1.6627213999657235 [0.15844806 2.98040173 3.3511544 ]


/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The fram

1.6484613842096978 [0.15873632 2.95311933 3.3218976 ]
1.6739358006169085 [0.15789675 3.02529469 3.36300272]
1.68529435910353 [0.1578204  3.02841606 3.36585069]
1.6892473507259225 [0.15763728 3.03136658 3.37025353]
1.6920930124855145 [0.15657022 3.04069234 3.37406018]
1.6947154247726153 [0.15598148 3.04755154 3.3770887 ]
1.0029607229188626 [0.87624561 1.13764828 1.15736335]
1.0027129478538954 [0.86832907 1.14468954 1.16327441]
1.0028333606390465 [0.87680714 1.14384936 1.16143739]
1.0046194535296034 [0.8700756  1.15326111 1.16565699]
1.0036489767977659 [0.8701099  1.14990466 1.16500597]
1.0050715395635088 [0.8666663  1.15452497 1.16597398]
1.0052860248206876 [0.86265607 1.1551963  1.1659966 ]
1.005603836966791 [0.8624556  1.15636892 1.16633965]
1.0060999705381006 [0.86302291 1.15701564 1.16655451]
1.0062245043971838 [0.86186183 1.15752288 1.16664349]


/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The fram